## Ejercicio/Tarea
### Eduardo Martínez - 175921

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?

In [1]:
import pandas as pd

from dask import persist, compute
from dask_ml.model_selection import GridSearchCV
from dask_ml.linear_model import LinearRegression
import dask.dataframe as dd

from timeit import default_timer as tic

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

Se procesará el set de datos en `pandas` pues se obtendrán variables (features) a partir de las existentes y se eliminarán algunas otras. Después de la limpieza se convertirá en un dataframe de `dask`.

In [2]:
## Como en la clase, se llamará al dataset el nombre trips_df

trips_df = pd.read_csv("/data/trips.csv")
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


Como se quiere la proporción de propina que se dejó en un viaje en taxi, sólo se considerarán tarifas positivas, i.e. `fare_amount > 0`

In [3]:
trips_df = trips_df[trips_df.fare_amount > 0]

De nuevo, como el objetivo es la proporción de propina de un viaje se definirá una nueva variable que es el porcentaje de propina: `porc_prop = tip_amount/fare_amount`

In [4]:
trips_df["porc_prop"] = trips_df.apply(lambda df: df["tip_amount"] / df["fare_amount"], axis=1)
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,porc_prop
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333


Se puede notar que la variable `car_type` es categórica. Para que se pueda aplicar a alguno de los algoritmos de ML requiere ser numérica; es por esto que se convertirá en dummy.

In [5]:
trips_df = pd.get_dummies(trips_df, columns=["car_type"])

trips_df.head()

,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,porc_prop,car_type_A,car_type_B
0,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091,1,0
1,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000,1,0
2,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333,1,0
3,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647,1,0
4,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333,1,0


Se convertirá a las variables `tpep_dropoff_datetime` y `tpep_pickup_datetime` en variables tipo fecha para poder hacer algunas operaciones entre ellas y obtener algunas variables derivadas

In [6]:
trips_df.tpep_dropoff_datetime = pd.to_datetime(trips_df.tpep_dropoff_datetime)
trips_df.tpep_pickup_datetime = pd.to_datetime(trips_df.tpep_pickup_datetime)
trips_df.head()

,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,porc_prop,car_type_A,car_type_B
0,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091,1,0
1,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000,1,0
2,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333,1,0
3,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647,1,0
4,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333,1,0


No se considerará a las variables `tpep_dropoff_datetime` y `tpep_pickup_datetime`, en vez de éstas se obtendrá la duración del viaje en minutos

In [7]:
def convierte_a_minutes(fecha):
    return (fecha.hour * 60 + fecha.minute)

trips_df["dur_viaje"] =  (pd.to_datetime(trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime)
                            .apply(convierte_a_minutes))

trips_df.head()

,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,porc_prop,car_type_A,car_type_B,dur_viaje
0,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091,1,0,19
1,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000,1,0,9
2,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333,1,0,10
3,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647,1,0,9
4,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333,1,0,9


NO se considerarán las variables `taxi_id` pues es simplemente un identificador del taxi, ni `tpep_dropoff_datetime` y `tpep_pickup_datetime` pues ya están representadas en `dur_viaje`. Además, se quitarán los renglones que tengan faltantes (NA).

In [8]:
label_features = ["porc_prop",
                  "fare_amount",
                  "passenger_count",
                  "trip_distance",
                  "car_type_A",
                  "car_type_B",
                  "dur_viaje"]

trips_df = trips_df[label_features].dropna(axis=0, how='all')

In [9]:
trips_df.head()

,porc_prop,fare_amount,passenger_count,trip_distance,car_type_A,car_type_B,dur_viaje
0,0.209091,22.0,1,6.90,1,0,19
1,0.000000,9.0,1,1.81,1,0,9
2,0.133333,7.5,1,0.96,1,0,10
3,0.117647,8.5,1,1.90,1,0,9
4,0.221333,7.5,1,1.00,1,0,9


Se puede obtener algunas métricas estándar de las variables del dataframe.

In [10]:
trips_df.describe()

,porc_prop,fare_amount,passenger_count,trip_distance,car_type_A,car_type_B,dur_viaje
count,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000
mean,0.130459,11.871227,1.688459,2.737321,0.419993,0.580007,12.210269
std,0.128037,10.338138,1.341322,3.294949,0.493584,0.493584,24.593648
min,0.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,6.500000,1.000000,1.000000,0.000000,0.000000,6.000000
50%,0.153846,9.000000,1.000000,1.660000,0.000000,1.000000,9.000000
75%,0.222222,13.000000,2.000000,3.000000,1.000000,1.000000,15.000000
max,3.600000,230.000000,9.000000,41.940000,1.000000,1.000000,1438.000000


Se puede observar que hay una duración de viaje muy grande con una distancia muy corta. Se ordenará de mayor a menor a partir de esta variable para hacer un poco más de exploración

In [11]:
trips_df.sort_values(by=['dur_viaje'], ascending=0).head(10)

,porc_prop,fare_amount,passenger_count,trip_distance,car_type_A,car_type_B,dur_viaje
4834,0.107407,13.5,1,2.73,1,0,1438
667,0.000000,5.0,1,0.53,0,1,1427
1892,0.056667,6.0,5,1.29,0,1,872
1335,0.086580,115.5,1,25.42,1,0,137
4125,0.237069,92.8,1,21.40,1,0,120
2467,0.134615,52.0,2,16.80,0,1,108
3541,0.223077,52.0,1,17.50,1,0,86
3657,0.192308,52.0,1,17.20,1,0,84
3089,0.115385,52.0,1,17.40,1,0,83
5745,0.224000,50.0,1,8.80,1,0,80


Se puede observar que las tres duraciones mayores parecen inconsistentes con la distancia recorrida. Esto sugiere una inconsistencia en el registro y por lo tanto no se considerarán

In [12]:
trips_df = trips_df[trips_df.dur_viaje < 870]
trips_df.describe()

,porc_prop,fare_amount,passenger_count,trip_distance,car_type_A,car_type_B,dur_viaje
count,9190.000000,9190.000000,9190.000000,9190.000000,9190.000000,9190.000000,9190.000000
mean,0.130484,11.872436,1.688248,2.737719,0.420022,0.579978,11.807617
std,0.128048,10.339381,1.341058,3.295372,0.493589,0.493589,9.239936
min,0.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,6.500000,1.000000,1.000000,0.000000,0.000000,6.000000
50%,0.153846,9.000000,1.000000,1.660000,0.000000,1.000000,9.000000
75%,0.222222,13.000000,2.000000,3.000000,1.000000,1.000000,15.000000
max,3.600000,230.000000,9.000000,41.940000,1.000000,1.000000,137.000000


Finalmente, se creará un dataframe de dask para hacer el análisis

In [13]:
trips_dask = dd.from_pandas(trips_df, npartitions=10)

Y se puede verificar de qué tipo es este dataframe

In [14]:
type(trips_dask)

dask.dataframe.core.DataFrame

In [15]:
trips_train, trips_test = trips_dask.random_split([0.7, 0.3], random_state=2)

In [16]:
feat = ["fare_amount",
        "passenger_count",
        "trip_distance",
        "car_type_A",
        "car_type_B",
        "dur_viaje"]

X_train, y_train = trips_train[feat], trips_train['porc_prop'] 
X_test, y_test = trips_test[feat], trips_test['porc_prop']

In [17]:
X_train = X_train.values.compute()
y_train = y_train.values.compute()

**Importante:** Se considerará una regresión lineal donde se variará el nivel de regularización C y el de tolerancia; además de se considerará si el modelo tiene ordenada al origen o no (i.e. si tiene `beta_0`). La elección de estos parámetros se debe a que dask es más limitado en su catálogo de modelos entonces para poder hacerlo comparable con SciKitLearn se eligió éste.

+ En `dask_ml` el modelo de regresión con regularización se ejecuta con el comando `LinearRegression()`.

+ En `sklearn` el modelo de regresión con regularización se ejecuta con el comando `Ridge()`.

**Observación:** Intenté un XGBoost que también podría compararase con el de SciKitLearn pero no pude hacer la instalación correspodiente; la instrucción

`!pip install xgboost`

me marcó error. Por esta razón consideré una simple regresión.

In [18]:
# Modelo
reglin = LinearRegression()

# Sólo se considerará un pipeline con la regresión lineal
pipeline = Pipeline([('reglin', LinearRegression())])

# Parámetros que se variararán en la regresión
grid_params = [{'reglin__C': [2, 4, 10],
                'reglin__tol': [1e-1, 1e-2, 1e-3],
                'reglin__fit_intercept': [0,1]}]

# Construcción del grid search:
jobs = -1
grid = GridSearchCV(estimator=pipeline,
            param_grid=grid_params,
            cv=5, 
            n_jobs=jobs)

# Función que recorre el grid y los modelos
def busqueda_dask(grids, models, X, y):
    best_acc = 1 ## La exactitud perfecta es 1 por eso es el parámetro de referencia
    best_gs = ''        
    
    for ind, gs in enumerate(grids):
        print('Ajustando modelo ' + models[ind] + ' con dask; no desespere...')

        # Fit model:
        gs.fit(X, y)
        gs.predict(X)    
        
        # Score:
        score = gs.score(X, y)
        print('Score: %.6f' % score + "\n")
        
        # Best model:
        if score < best_acc:
            best_acc = score
            best_gs = gs
    
    return(best_gs)

## Ejecución en `dask` secuencial

In [19]:
grids = [grid]
models = ['LinearRegression']

In [20]:
%%time
mod_regr_lin = busqueda_dask(grids, models, X_train, y_train)

Ajustando modelo LinearRegression con dask; no desespere...
Score: 0.015261

CPU times: user 1min 5s, sys: 1min 26s, total: 2min 31s
Wall time: 38.8 s


In [21]:
mod_regr_lin.best_params_

{'reglin__C': 2, 'reglin__fit_intercept': 0, 'reglin__tol': 0.1}

## Ejecución en `dask` distribuido

In [22]:
from dask.distributed import Client
client = Client("scheduler:8786")

In [23]:
%%time
mod_regr_lin_distribuido = busqueda_dask(grids, models, X_train, y_train)

Ajustando modelo LinearRegression con dask; no desespere...
Score: 0.015261

CPU times: user 393 ms, sys: 72.5 ms, total: 466 ms
Wall time: 1min 53s


In [24]:
mod_regr_lin_distribuido.best_params_

{'reglin__C': 2, 'reglin__fit_intercept': 1, 'reglin__tol': 0.01}

## Ejecución en `sklearn`

In [25]:
# Modelo
reglin = Ridge()

# Sólo se considerará un pipeline con la regresión lineal
pipeline = Pipeline([('reglin', Ridge())])

# Parámetros que se variararán en la regresión
grid_params = [{'reglin__alpha': [2, 4, 10],
                'reglin__tol': [1e-1, 1e-2, 1e-3],
                'reglin__fit_intercept': [0,1]}]

# Construcción del grid search:
jobs = -1
grid = GridSearchCV(estimator=pipeline,
            param_grid=grid_params,
            cv=5, 
            n_jobs=jobs)

# Función que recorre el grid y los modelos
def busqueda_sklearn(grids, models, X, y):
    best_acc = 1 ## La exactitud perfecta es 1 por eso es el parámetro de referencia
    best_gs = ''        
    
    for ind, gs in enumerate(grids):
        print('Ajustando modelo ' + models[ind] + ' con SciKitLearn; no desespere...')

        # Fit model:
        gs.fit(X, y)
        gs.predict(X)    
        
        # Score:
        score = gs.score(X, y)
        print('Score: %.6f' % score + "\n")
        
        # Best model:
        if score < best_acc:
            best_acc = score
            best_gs = gs
    
    return(best_gs)

Ya se tiene la base de datos preparada en `pandas` así que simplemente se volverá a dividir en entrenamiento y prueba. Posteriormente se aplicará el grid que se definió.

In [26]:
X_train_skl, X_test_skl, y_train_skl, y_test_skl = train_test_split(
    trips_df[feat],trips_df["porc_prop"],test_size=0.3)

In [27]:
grids = [grid]
models = ['Ridge']

In [28]:
%%time
mod_regr_lin_sklearn = busqueda_sklearn(grids, models, X_train_skl, y_train_skl)

Ajustando modelo Ridge con SciKitLearn; no desespere...
Score: 0.000360

CPU times: user 114 ms, sys: 18.6 ms, total: 133 ms
Wall time: 738 ms


In [29]:
mod_regr_lin_sklearn.best_params_

{'reglin__alpha': 10, 'reglin__fit_intercept': 1, 'reglin__tol': 0.1}

### En resumen:

+ Con `dask` secuencial: Sin intercepto, tolerancia (numérica/exactiud) del 1\% y regularización 1/2. Tiempo de ejecución: 38.8 s.

+ Con `dask` distribuido: Con intercepto, tolerancia (numérica/exactiud) del 1\% y regularización 1/4. Tiempo de ejecución: 1min 53s.

+ Con `sklearn`: Con intercepto, tolerancia (numérica/exactiud) del 10\% y regularización 1/10. Tiempo de ejecución: 738 ms.